<a href="https://colab.research.google.com/github/maitysuvo19/News-Articles-Classification/blob/main/Real_news_classification_BERT_ktrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#basic imports
import os
import pandas as pd
import numpy as np

Preparing the data

In [ ]:
!unzip news.zip -d news

Archive:  news.zip
   creating: news/news article/News Articles/
   creating: news/news article/News Articles/business/
  inflating: news/news article/News Articles/business/001.txt  
  inflating: news/news article/News Articles/business/002.txt  
  inflating: news/news article/News Articles/business/003.txt  
  inflating: news/news article/News Articles/business/004.txt  
  inflating: news/news article/News Articles/business/005.txt  
  inflating: news/news article/News Articles/business/006.txt  
  inflating: news/news article/News Articles/business/007.txt  
  inflating: news/news article/News Articles/business/008.txt  
  inflating: news/news article/News Articles/business/009.txt  
  inflating: news/news article/News Articles/business/010.txt  
  inflating: news/news article/News Articles/business/011.txt  
  inflating: news/news article/News Articles/business/012.txt  
  inflating: news/news article/News Articles/business/013.txt  
  inflating: news/news article/News Articles/bus

In [ ]:
# Step 1 - Get the file details
directory = []
file = []
title = []
text = []
label = []
datapath = '/content/news/news article' 
for dirname, _ , filenames in os.walk(datapath):
    #print('Directory: ', dirname)
    #print('Subdir: ', dirname.split('/')[-1])
    # remove the Readme.txt file
    # will not find file in the second iteration so we skip the error
    try:
        filenames.remove('README.TXT')
    except:
        pass
    for filename in filenames:
        directory.append(dirname)
        file.append(filename)
        label.append(dirname.split('/')[-1])
        #print(filename)
        fullpathfile = os.path.join(dirname,filename)
        with open(fullpathfile, 'r', encoding="utf8", errors='ignore') as infile:
            intext = ''
            firstline = True
            for line in infile:
                if firstline:
                    title.append(line.replace('\n',''))
                    firstline = False
                else:
                    intext = intext + ' ' + line.replace('\n','')
            text.append(intext)

#    

In [ ]:
fulldf = pd.DataFrame(list(zip(directory, file, title, text, label)), 
               columns =['directory', 'file', 'title', 'text', 'label'])

df = fulldf.filter(['text','label'], axis=1)

print("FullDf : ", fulldf.shape)
print("DF : ", df.shape)

FullDf :  (2225, 5)
DF :  (2225, 2)


In [ ]:
df.head(5)

,text,label
0,Scotland's first minister has told a group o...,politics
1,Charles Kennedy is far too canny to make any...,politics
2,"Tony Blair has told Labour supporters he's ""...",politics
3,Two of Britain's big trade unions could merg...,politics
4,UK Foreign Secretary Jack Straw has defended...,politics


In [ ]:
df = pd.concat([df, df.label.astype('str').str.get_dummies()], axis=1, sort=False)
df = df[['text','sport','business','politics','tech','entertainment']]
df.head(5)

,text,sport,business,politics,tech,entertainment
0,Scotland's first minister has told a group o...,0,0,1,0,0
1,Charles Kennedy is far too canny to make any...,0,0,1,0,0
2,"Tony Blair has told Labour supporters he's ""...",0,0,1,0,0
3,Two of Britain's big trade unions could merg...,0,0,1,0,0
4,UK Foreign Secretary Jack Straw has defended...,0,0,1,0,0


Bert ktrain

In [ ]:
!pip install  -q ktrain

     |████████████████████████████████| 25.3MB 126kB/s 
     |████████████████████████████████| 6.8MB 50.3MB/s 
     |████████████████████████████████| 983kB 55.5MB/s 
     |████████████████████████████████| 266kB 55.2MB/s 
     |████████████████████████████████| 1.9MB 52.4MB/s 
     |████████████████████████████████| 1.2MB 52.8MB/s 
     |████████████████████████████████| 471kB 52.2MB/s 
     |████████████████████████████████| 901kB 50.9MB/s 
     |████████████████████████████████| 3.3MB 52.2MB/s 


In [ ]:
#Importing
import ktrain
from ktrain import text

In [ ]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_df(df, 
                                                                   'text', # name of column containing review text
                                                                   label_columns=['sport','business','politics','tech','entertainment'],
                                                                   maxlen=500, 
                                                                   preprocess_mode='bert',
                                                                   val_pct=0.1,
                                                                   )

['sport', 'business', 'politics', 'tech', 'entertainment']
      sport  business  politics  tech  entertainment
1188      0         1         0     0              0
162       0         0         1     0              0
1872      0         0         0     0              1
504       1         0         0     0              0
668       1         0         0     0              0
['sport', 'business', 'politics', 'tech', 'entertainment']
      sport  business  politics  tech  entertainment
920       1         0         0     0              0
68        0         0         1     0              0
1994      0         0         0     0              1
1836      0         0         0     1              0
779       1         0         0     0              0
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


# STEP 2: Loading a pre trained BERT and wrapping it in a ktrain.learner object

In [ ]:
model = text.text_classifier('bert', (x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model,train_data=(x_train, y_train), val_data=(x_test, y_test), batch_size=6)

Is Multi-Label? False
maxlen is 500
done.


# STEP 3: Training and Tuning the model's parameters

In [ ]:
learner.fit_onecycle(2e-5, 4)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/4
334/334 [==============================] - 364s 1s/step - loss: 1.2441 - accuracy: 0.5311 - val_loss: 0.1531 - val_accuracy: 0.9462
Epoch 2/4
334/334 [==============================] - 345s 1s/step - loss: 0.0745 - accuracy: 0.9810 - val_loss: 0.0774 - val_accuracy: 0.9821
Epoch 3/4
334/334 [==============================] - 345s 1s/step - loss: 0.0279 - accuracy: 0.9925 - val_loss: 0.0762 - val_accuracy: 0.9821
Epoch 4/4
334/334 [==============================] - 346s 1s/step - loss: 0.0060 - accuracy: 0.9993 - val_loss: 0.0759 - val_accuracy: 0.9776


# Inspecting Misclassifications

In [ ]:
learner.view_top_losses(n=5, preproc=preproc)

----------
id:59 | loss:5.39 | true:tech | pred:business)

[CLS] asia is set to drive global media growth to 2008 and beyond , with china and india filling the two top spots , analysts have predicted . japan , south korea and singapore will also be strong players , but china ' s demographics give it the edge , a media conference in london heard . the world ' s most populous country - population 1 . 3 ##bn - now has about 200 million middle - class consumers . forty per cent fall in the key 16 to 35 - year - old demographic . as a result , it is attracting huge foreign investment in media and communications , analysts told the financial times new media and broadcasting conference last week . interest in china among international media groups has surged in recent months after beijing issued rules allowing foreign investment in joint - venture television , radio and film production companies . news corporation , via ##com and sony pictures are among the big names involved in joint venture

# Making Predictions on New Data

In [ ]:
p = ktrain.get_predictor(learner.model, preproc)

In [ ]:
p.predict("Sachin made hightest run")

'sport'

# Save our Predictor for Later Deployment

In [ ]:
# save model for later use
p.save('/content/tmp/news_classifier')

In [ ]:
# reload from disk
#p = ktrain.load_predictor('/content/tmp/news_classifier')

In [ ]:
# still works as expected after reloading from disk
#p.predict("Sachin made hightest run")